# 实验 3.4 - 学员笔记本

## 概述

本实验是模块 3 引导式实验的延续。

在本实验中，您会将数据拆分为三个独立的数据集：

- *训练集* - 将用于训练模型。
- *验证集* - 训练期间将使用它来验证模型。
- *测试集* - 将保留该数据集，用于在训练模型后生成指标。您将在接下来的实验中使用此数据集。

拆分数据后，您将使用 Amazon SageMaker 训练 XGBoost 模型。


## 业务场景简介

您在一家医疗保健服务提供商工作，并希望改善骨科患者的异常检测。

您的任务是利用机器学习 (ML) 解决此问题。您可以使用包含六个生物力学特征且目标为*正常*或*异常*的数据集。您可以使用此数据集训练 ML 模型，以预测患者是否会出现异常。


## 关于该数据集

该生物医学数据集由 Henrique da Mota 博士在法国里昂 Médico-Chirurgical de Réadaptation des Massues 中心的整形外科应用研究组 (GARO) 实习期间创建。这些数据分到两个不同但相关的分类任务中。

第一项任务是将患者归类为以下三类之一： 

- *正常*（100 名患者）
- *椎间盘疝*（60 名患者）
- *脊椎滑脱*（150 名患者）

对于第二个任务，则是将*椎间盘疝*和*脊椎滑脱*合并为一个类别，标记为*异常*。因此，在第二个任务中，患者属于以下两个类别之一：*正常*（100 名患者）或*异常*（210 名患者）。


## 属性信息：

数据集中的每名患者都由六个生物力学属性表示，这些属性（顺序如下）是根据骨盆和腰椎的形状和方向得出的： 

- 骨盆入射角
- 骨盆倾斜角
- 腰椎前凸角
- 骶骨倾斜角
- 骨盆半径
- 脊椎滑脱等级

以下约定用于分类标签： 
- 椎间盘疝 (DH)
- 脊椎滑脱 (SL)
- 正常 (NO) 
- 异常 (AB)


有关此数据集的更多信息，请参阅[脊柱数据集网页](http://archive.ics.uci.edu/ml/datasets/Vertebral+Column)。


## 数据集属性

该数据集来自：
Dua, D. 和 Graff, C.（2019 年）。UCI 机器学习存储库 (http://archive.ics.uci.edu/ml)。加州尔湾市：加利福尼亚大学信息与计算机科学学院。


# 实验设置
由于此解决方案分散在模块中的多个实验中，因此您需要执行以下单元格中的内容，以便加载数据。

## 导入数据

通过执行以下单元格中的内容，将导入数据并让数据可供使用。

**注意**：以下单元格中的内容代表以前的实验中的关键步骤。

In [1]:
import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff
import boto3

In [2]:
f_zip = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'
r = requests.get(f_zip, stream=True)
Vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))
Vertebral_zip.extractall()

In [3]:
data = arff.loadarff('column_2C_weka.arff')
df = pd.DataFrame(data[0])

In [4]:
class_mapper = {b'Abnormal':1,b'Normal':0}
df['class']=df['class'].replace(class_mapper)

# 步骤 1：探索数据
您首先将快速了解数据集中的数据。

为了充分利用本实验，在执行单元格中的内容之前，请仔细阅读说明和代码。花点时间进行实验！

首先，使用 **shape** 来检查行数和列数。

In [5]:
df.shape

(310, 7)

接下来，获取列的列表。

In [6]:
df.columns

Index(['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis', 'class'],
      dtype='object')

您可以看到六个生物力学特征，目标列名为 *class*。


# 步骤 2：准备数据

在本实验中，您需要将数据拆分为三个数据集。

通过在互联网上搜索，您可以找到许多不同的数据集拆分方式。您找到的许多代码範例可能会将数据集分为*目标*和*特征*。然后，这两个数据集又分别拆分为三个子集，从而总共产生了六个数据集。

## 移动目标列位置

XGBoost 要求训练数据存放在单个文件中。该文件的目标值必须在第一列。

获取目标列并将其移到第一个位置。

In [7]:
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

您应该可以看到 **class** 现在是第一列了。

In [8]:
df.columns

Index(['class', 'pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis'],
      dtype='object')

## 拆分数据

首先将数据集分为两个数据集。您将使用其中一个数据集进行训练，并将再次拆分另一个数据集以用于验证和测试。

您将使用 *scikit-learn 库*中的 *train_test_split 函数*，该库是免费的 Python 机器学习库。它拥有许多算法和有用的函数，包括您将使用的这个。

- 有关该函数的更多信息，请参阅 [Train_test_split 文档](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)。
 - 有关 scikit-learn 的更多信息，请参阅 [scikit-learn 指南](https://scikit-learn.org/stable/)

因为数据量并不多，所以您需要确保拆分的数据集包含每个类的代表性数量。因此，您将使用 *stratify* 开关。最后，您将使用一个随机数，以便可以重复进行拆分。

In [9]:
from sklearn.model_selection import train_test_split
train, test_and_validate = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])

接下来，将 *test_and_validate* 数据集拆分成两个相等的部分。

In [10]:
test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['class'])

检查三个数据集。

In [11]:
print(train.shape)
print(test.shape)
print(validate.shape)

(248, 7)
(31, 7)
(31, 7)


现在，检查类的分布。

In [12]:
print(train['class'].value_counts())
print(test['class'].value_counts())
print(validate['class'].value_counts())

class
1    168
0     80
Name: count, dtype: int64
class
1    21
0    10
Name: count, dtype: int64
class
1    21
0    10
Name: count, dtype: int64


## 将数据上传到 Amazon S3

XGboost 将从 Amazon Simple Storage Service (Amazon S3) 加载用于训练的数据。因此，您必须将数据写入逗号分隔值 (CSV) 文件，然后将文件上传到 Amazon S3。

首先为 S3 存储桶设置一些变量，然后创建一个将 CSV 文件上传到 Amazon S3 的函数。您可以重复使用该函数。

首先浏览函数。

请注意以下这行：

`dataframe.to_csv(csv_buffer, header=False, index=False)`

此行将 pandas DataFrame（已传递到函数中）写入名为 *csv_buffer* 的 I/O 缓冲区。通过使用缓冲区，就可以避免在本地写入文件。

要阻止列标题写出，请使用 `header=False`。要阻止输出 pandas 索引，请使用 `index=False`。

要将 csv_buffer 作为对象写入 Amazon S3，请对 `object`（`bucket` 的一个属性）使用 `put` 操作。


In [13]:
bucket='c130335a3301600l8062774t1w632558395917-labbucket-uq8nhb2ic15j'

prefix='lab3'

train_file='vertebral_train.csv'
test_file='vertebral_test.csv'
validate_file='vertebral_validate.csv'

import os

s3_resource = boto3.Session().resource('s3')
def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False)
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

使用您刚才创建的函数上传三个数据集。

In [14]:
upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

# 步骤 3：训练模型

现在，数据已经存储到了 Amazon S3 中，您可以开始训练模型了。

第一步是获取 XGBoost 容器 URI。

In [15]:
import boto3
from sagemaker.image_uris import retrieve
container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


接下来，您需要为模型设置一些*超参数*。因为这是您第一次训练模型，所以开始时可以使用一些值。

In [16]:
hyperparams={"num_round":"42",
             "eval_metric": "auc",
             "objective": "binary:logistic"}

使用 **estimator** 函数设置模型。以下是一些需要关注的参数：

- **instance_count** - 定义将用于训练的实例数。您将使用*一个*实例。
- **instance_type** - 定义用于训练的实例类型。这里使用的是 *ml.m4.xlarge*。


In [17]:
import sagemaker
s3_output_location="s3://{}/{}/output/".format(bucket,prefix)
xgb_model=sagemaker.estimator.Estimator(container,
                                       sagemaker.get_execution_role(),
                                       instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())

估算器需要*通道*才能将数据输入模型。对于训练，将使用 *train_channel* 和 *validate_channel*。

In [18]:
train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

运行 **fit** 将训练模型。

**注意**：这一过程耗时最多 5 分钟。

In [19]:
xgb_model.fit(inputs=data_channels, logs=False)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-10-22-12-30-11-475



2024-10-22 12:30:14 Starting - Starting the training job..
2024-10-22 12:30:28 Starting - Preparing the instances for training....
2024-10-22 12:30:54 Downloading - Downloading input data.....
2024-10-22 12:31:24 Downloading - Downloading the training image.........
2024-10-22 12:32:15 Training - Training image download completed. Training in progress...
2024-10-22 12:32:31 Uploading - Uploading generated training model..
2024-10-22 12:32:44 Completed - Training job completed


训练完成后，您就可以测试和评估模型了。但我们在之后的实验中才会讨论测试和验证部分。

# 恭喜！

您已经完成了本实验，现在可以按照实验指南中的说明结束本实验。